In [4]:
#!/usr/bin/python3
import re
import subprocess
import os
os.getcwd()
os.chdir("data")
#perform blastp of pair genome with each other to find orthologus gene or region

In [ ]:
#make blastdb for First query genome

subprocess.call(['makeblastdb', '-in', 'Rossellomorea_vietnamemsis_RAR_WA_32.faa', '-dbtype', 'prot', '-parse_seqids', '-out', 'Rossellomorea_vietnamemsis_RAR_WA_32'], stdout=subprocess.PIPE)

In [ ]:
# Run BLASTP for query genome against subject genome
subprocess.call(["blastp", "-query", 'MCCB_382.faa', "-subject", 'Rossellomorea_vietnamemsis_RAR_WA_32.faa', "-out", "MCCB_382_vs_Rossellomorea_vietnamemsis_RAR_WA_32.tab", "-evalue", "1e-5", "-outfmt", "6 std qlen",'-max_target_seqs', '1'], stdout=subprocess.PIPE)


In [ ]:
#similarly make blastdb for First query genome ()

subprocess.call(['makeblastdb', '-in', 'MCCB_382.faa', '-dbtype', 'prot', '-parse_seqids', '-out', 'MCCB_382'], stdout=subprocess.PIPE)


In [ ]:
# Run BLASTP for subject genome against query genome
subprocess.call(["blastp", "-query", 'Rossellomorea_vietnamemsis_RAR_WA_32.faa', "-subject", 'MCCB_382.faa', "-out", "Rossellomorea_vietnamemsis_RAR_WA_32_vs_MCCB_382.tab", "-evalue", "1e-5", "-outfmt", "6 std qlen",'-max_target_seqs', '1'], stdout=subprocess.PIPE)


In [7]:
# read the blast output file for both genome
import pandas as pd
query_results = pd.read_csv('MCCB_382_vs_Rossellomorea_vietnamemsis_RAR_WA_32.tab', sep="\t", header=None)
print(len(query_results[1]))
query_results.head()

4184


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,KBBLJFAO_00001,CNFHIFKK_04266,96.732,153,5,0,1,153,1,153,7.300000e-108,301.0,153
1,KBBLJFAO_00002,CNFHIFKK_04265,92.857,182,13,0,1,182,1,182,6.550000e-127,351.0,182
2,KBBLJFAO_00003,CNFHIFKK_04264,96.389,360,13,0,1,360,1,360,0.000000e+00,708.0,360
3,KBBLJFAO_00004,CNFHIFKK_04263,99.632,815,3,0,1,815,1,815,0.000000e+00,1639.0,815
4,KBBLJFAO_00005,CNFHIFKK_04262,99.563,458,2,0,1,458,1,458,0.000000e+00,934.0,458


In [8]:
# read the blast output file 
subject_results = pd.read_csv('Rossellomorea_vietnamemsis_RAR_WA_32_vs_MCCB_382.tab', sep="\t", header=None)
print(len(subject_results[1]))
print(subject_results)

4157
                  0               1       2    3    4   5    6    7    8   \
0     CNFHIFKK_00001  KBBLJFAO_00279  99.317  439    3   0    1  439    1   
1     CNFHIFKK_00002  KBBLJFAO_00280  99.010  101    1   0    1  101    1   
2     CNFHIFKK_00003  KBBLJFAO_00281  97.764  313    7   0    1  313    1   
3     CNFHIFKK_00005  KBBLJFAO_01127  69.716  634  183   4    1  632    1   
4     CNFHIFKK_00006  KBBLJFAO_01128  84.259  648  100   1    1  646    1   
...              ...             ...     ...  ...  ...  ..  ...  ...  ...   
4152  CNFHIFKK_04656  KBBLJFAO_03387  29.720  286  184   3  311  579   90   
4153  CNFHIFKK_04657  KBBLJFAO_03292  50.714  420  181   7    1  417    1   
4154  CNFHIFKK_04659  KBBLJFAO_02984  24.595  309  207  10  102  391   92   
4155  CNFHIFKK_04660  KBBLJFAO_04244  22.967  209  140   5  235  430  188   
4156  CNFHIFKK_04661  KBBLJFAO_04242  60.500  400  152   3    2  397    3   

       9              10      11   12  
0     439   0.000000e+00   890

In [7]:
#count the nymber of sequence in each fasta file
#using re 
pattern = r">(\S+)"

# Processing MCCB_382.faa
with open("MCCB_382.faa", "r") as f:
    query_data = f.read()

query_matches = re.findall(pattern, query_data)
query_counts = {}
for query_id in set(query_matches):
    query_count = query_matches.count(query_id)
    query_counts[query_id] = query_count
print(len(query_counts))


# Processing Rossellomorea_vietnamemsis_RAR_WA_32.faa 
#using defaultdict
from collections import defaultdict

subject_counts = defaultdict(int)

with open("Rossellomorea_vietnamemsis_RAR_WA_32.faa", "r") as f:
    for line in f:
        if line.startswith('>'):
            query_id = line.strip()[1:]#remove> and extract id 
            subject_counts[query_id] += 1

print(len(subject_counts))

#we can also use display function from IPython    
from IPython.display import display
display(len(subject_counts))

4698
4517


4517

In [2]:
#or simple way to count number of >
query_counts = 0  # Initialize a counter

with open("MCCB_382.faa", "r") as f:
    for line in f:
        if line.startswith('>'):
            query_counts += 1

print(query_counts)


4698


In [16]:
# Create query and subject coverage columns in both dataframes
query_results['qcov'] = query_results[3] / query_results[12]
subject_results['qcov'] = subject_results[3] / subject_results[12]

# Apply your custom filtering logic
threshold_ident = 40
threshold_qcov = 0.5

# Filter based on conditions using integer index
query_conserved_protein = query_results[(query_results[2] >= threshold_ident) & (query_results['qcov'] >= threshold_qcov)]
subject_conserved_protein = subject_results[(subject_results[2] >= threshold_ident) & (subject_results['qcov'] >= threshold_qcov)]

#or by using index
# Filter based on conditions using .iloc
#query_conserved_protein = query_results[(query_results.iloc[:, 2] >= threshold_ident) & (query_results.iloc[:, 13] >= threshold_qcov)]
#subject_conserved_protein = subject_results[(subject_results.iloc[:, 2] >= threshold_ident) & (subject_results.iloc[:, 13] >= threshold_qcov)]


In [17]:
# sometime we need to change as integer type 
#query_results[10] = query_results[10].astype(float)
#subject_results[10] = subject_results[10].astype(float)

In [19]:
#Filter duplicates based on columns 0 and 1
query_conserved_protein = query_conserved_protein.drop_duplicates(subset=[0, 1])
subject_conserved_protein= subject_conserved_protein.drop_duplicates(subset=[0, 1])

print(len(query_conserved_protein))

print(len(subject_conserved_protein))


3931
3914


In [22]:
# Calculate the final percentage
total_queries = len(query_counts)+len(subject_counts)
hit_sum = len(query_conserved_protein[1]) + len(subject_conserved_protein[1])
percentage = (hit_sum / total_queries) * 100

print(percentage)
print(f"POCP Percentage: {percentage:.2f}%")

85.13293543136191
POCP Percentage: 85.13%
